In [1]:
import torch

print("CUDA khả dụng:", torch.cuda.is_available())  
if torch.cuda.is_available():
    print("Tên GPU:", torch.cuda.get_device_name(0))
    print("Phiên bản CUDA:", torch.version.cuda)


CUDA khả dụng: True
Tên GPU: NVIDIA GeForce MX350
Phiên bản CUDA: 11.8


# 1. Huggingface models

In [2]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain. llms.huggingface_pipeline import HuggingFacePipeline

# 1.1 Load model from huggingface

In [3]:
model_name: str = "microsoft/phi-2"

nf4_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_use_double_quant=True,
bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM. from_pretrained(
model_name,
quantization_config=nf4_config,
low_cpu_mem_usage=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
tokenizer = AutoTokenizer. from_pretrained(model_name)
max_new_token = 1024

model_pipeline = pipeline(
"text-generation",
model=model,
tokenizer=tokenizer,
max_new_tokens=max_new_token,
pad_token_id=tokenizer.eos_token_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
gen_kwargs = {
"temperature": 0.6
}
llm = HuggingFacePipeline(
pipeline=model_pipeline,
model_kwargs=gen_kwargs)

C:\Users\ngchn\AppData\Local\Temp\ipykernel_7996\519883150.py:4: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(


In [6]:
llm.invoke("Hello, how are you?")

'Hello, how are you?\n\nI hope this email finds you well. I am writing to you as a fellow alumni of our high school. I just wanted to say hello and see how you are doing.\n\nI am currently working as a software engineer at a tech company in San Francisco. I enjoy my job and I am always learning new things. I also got married last year and we have a cute dog named Milo.\n\nWhat about you? What have you been up to? Are you still pursuing your passion for photography? Have you traveled to any new places? I would love to hear from you and catch up on our lives.\n\nPlease reply when you have some time. I miss you and I hope we can reconnect soon.\n\nBest regards,\nYour name\n'

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate. from_template(
"""Instruct: {prompt}\nOutput:"
"""
)

user_prompt = "Write a detailed analogy between mathematics and a lighthouse."
messages = prompt_template. format(prompt=user_prompt)

In [8]:
print(messages)

Instruct: Write a detailed analogy between mathematics and a lighthouse.
Output:"



In [9]:
output = llm.invoke(messages)
print(output)

Instruct: Write a detailed analogy between mathematics and a lighthouse.
Output:"

In mathematics, a lighthouse is like a guiding light that helps sailors navigate through the vast ocean of numbers and equations. Just as a lighthouse emits a strong beam of light to guide ships to safety, mathematics provides us with a set of tools and techniques to navigate through complex problems and find solutions.

Just as a lighthouse has a clear and focused beam of light, mathematics provides us with a clear and logical framework to analyze and solve problems. It helps us to organize our thoughts and approach problems in a systematic manner.

Just as a lighthouse is built on a solid foundation, mathematics is built on a foundation of fundamental concepts and principles. These concepts serve as the building blocks for more advanced mathematical ideas and allow us to explore and understand the world around us.

Just as a lighthouse is constantly monitored and maintained, mathematics requires contin

# 2. ChatPrompt template

In [10]:
from langchain_core.prompts import ChatPromptTemplate

In [11]:
messages = [
("system", "You are a helpful AI bot. Your name is {bot_name}."),
("user", "Hi!"),
("assistant", "Hello. How can I help you today?"),
("user", "{user_input}"),
]

In [12]:
chat_template = ChatPromptTemplate.from_messages(messages)

In [13]:
chat_template. format_messages(
bot_name="Halo",
user_input="Do you have mayonnaise recipes?") 

[SystemMessage(content='You are a helpful AI bot. Your name is Halo.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello. How can I help you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Do you have mayonnaise recipes?', additional_kwargs={}, response_metadata={})]

In [14]:
print(messages)

[('system', 'You are a helpful AI bot. Your name is {bot_name}.'), ('user', 'Hi!'), ('assistant', 'Hello. How can I help you today?'), ('user', '{user_input}')]
